In [1]:
# import libraries
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import shuffle
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [2]:
# authenticate to GCS account
from google.colab import auth
auth.authenticate_user()

In [6]:
# download data from GCS
!gsutil cp 'gs://tesis_devi/twitter_4000.csv' ./

Copying gs://tesis_devi/twitter_4000.csv...
/ [1 files][458.4 KiB/458.4 KiB]                                                
Operation completed over 1 objects/458.4 KiB.                                    


In [7]:
# read and shuffle data
data = pd.read_csv('twitter_4000.csv')
data = shuffle(data, random_state=21)

data.head(3)

,tweet,tag
3132,euh ouai le rapport avec camus si vous navez ...,france
2299,hah tadi kamu post di holy kah kok aku gak nyadar,indonesia
224,be a better you for you,english


In [8]:
# create label
labels = [labels.split(',') for labels in data['tag'].values]
encoder = MultiLabelBinarizer()
encoded_label = encoder.fit_transform(labels)
num_label = len(encoded_label[0])

print(data['tweet'].values[0])

print(encoder.classes_)
print(encoded_label[0])

euh ouai le rapport avec camus  si vous navez pas compris la rfrence foucault je peux vous expliquer donc nous sommes daccord  facultuellement il y a changement de population culturelle  vous trouvez a bien  moi dramatique  cest l notre diffrence
['english' 'france' 'germany' 'indonesia']
[0 1 0 0]


In [9]:
# split data

train_size = int(len(data) * .8)
train_label = encoded_label[:train_size]
test_label = encoded_label[train_size:]

In [10]:
# tokenizer class
# keras for building BoW model
%%writefile preprocess.py

from tensorflow.keras.preprocessing import text

class TextPreprocessor(object):
  def __init__(self, vocab_size):
    self.vocab_size = vocab_size
    self.tokenizer = None

  def create_tokenizer(self, text_list):
    tokenizer = text.Tokenizer(num_words=self.vocab_size)
    tokenizer.fit_on_texts(text_list)
    self._tokenizer = tokenizer

  def transform_text(self, text_list):
    text_matrix = self._tokenizer.texts_to_matrix(text_list)
    return text_matrix

Writing preprocess.py


In [11]:
# implementing Bow

from preprocess import TextPreprocessor

train_tweet = data['tweet'].values[:train_size]
test_tweet = data['tweet'].values[train_size:]

VOCAB_SIZE = 200

processor = TextPreprocessor(VOCAB_SIZE)
processor.create_tokenizer(train_tweet)

body_train = processor.transform_text(train_tweet)
body_test = processor.transform_text(test_tweet)

print(train_tweet[0])
print(body_train[0])


euh ouai le rapport avec camus  si vous navez pas compris la rfrence foucault je peux vous expliquer donc nous sommes daccord  facultuellement il y a changement de population culturelle  vous trouvez a bien  moi dramatique  cest l notre diffrence
[0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [33]:
# create model
def create_model(vocab_size, num_tags):
  model = Sequential() # MLP
  model.add(Dense(50, input_shape=(vocab_size,), activation='relu')) # first layer / input
  model.add(Dense(25, activation='relu')) # hidden layer
  model.add(Dense(num_label, activation='sigmoid')) # output layer

  # set up loss function, weight update method, and value to show when running model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

model = create_model(VOCAB_SIZE, num_label)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 50)                10050     
                                                                 
 dense_4 (Dense)             (None, 25)                1275      
                                                                 
 dense_5 (Dense)             (None, 4)                 104       
                                                                 
Total params: 11,429
Trainable params: 11,429
Non-trainable params: 0
_________________________________________________________________


In [34]:
# train model
model.fit(body_train, train_label, epochs=3, batch_size=128, validation_split=0.1)

Train on 2880 samples, validate on 320 samples
Epoch 1/3
2880/2880 [==============================] - 0s 42us/sample - loss: 0.6477 - accuracy: 0.6817 - val_loss: 0.5717 - val_accuracy: 0.8109
Epoch 2/3
2880/2880 [==============================] - 0s 15us/sample - loss: 0.5054 - accuracy: 0.8043 - val_loss: 0.4263 - val_accuracy: 0.8172
Epoch 3/3
2880/2880 [==============================] - 0s 14us/sample - loss: 0.3636 - accuracy: 0.8518 - val_loss: 0.2831 - val_accuracy: 0.9070


`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


In [35]:
# evaluate model
model.evaluate(body_test, test_label, batch_size=128)

[0.2999786376953125, 0.8978125]

In [36]:
# save model
model.save('thesis_twitter_model.h5')